In [1]:
import pandas as pd # Thank god
from sklearn import linear_model
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error

import numpy as np

In [2]:
df_train = pd.read_csv('house-prices-advanced-regression-techniques\logtrain.csv').set_index('Id')
(df_train).head()
df_train_explan = df_train.drop(columns = 'SalePrice')
df_train_y = df_train['SalePrice']


df_bintrain = pd.get_dummies(df_train_explan.astype(str))

df_bintrain.head()

,MSSubClass_120,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,1,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,1,0


# Part 2: Naive Data Processing

In [3]:
for colname, coldata in df_train_explan.iteritems():
    print(f'{colname} has', len(set(coldata)), 'unique features')


MSSubClass has 15 unique features
MSZoning has 5 unique features
LotFrontage has 369 unique features
LotArea has 1073 unique features
Street has 2 unique features
Alley has 3 unique features
LotShape has 4 unique features
LandContour has 4 unique features
Utilities has 2 unique features
LotConfig has 5 unique features
LandSlope has 3 unique features
Neighborhood has 25 unique features
Condition1 has 9 unique features
Condition2 has 8 unique features
BldgType has 5 unique features
HouseStyle has 8 unique features
OverallQual has 10 unique features
OverallCond has 9 unique features
YearBuilt has 112 unique features
YearRemodAdd has 61 unique features
RoofStyle has 6 unique features
RoofMatl has 8 unique features
Exterior1st has 15 unique features
Exterior2nd has 16 unique features
MasVnrType has 5 unique features
MasVnrArea has 335 unique features
ExterQual has 4 unique features
ExterCond has 5 unique features
Foundation has 6 unique features
BsmtQual has 5 unique features
BsmtCond has 5

In [4]:
# PART 2 QUESTION 4, TRAIN THE MODEL

from math import log, sqrt
train = [s.strip().split(",") for s in open('house-prices-advanced-regression-techniques\my_train.csv').readlines()]
train = train[1:]
dev = [s.strip().split(",") for s in open('house-prices-advanced-regression-techniques\my_dev.csv').readlines()]
dev = dev[1:]
test = [s.strip().split(",") for s in open('house-prices-advanced-regression-techniques\my_test.csv').readlines()]
test = test[1:]

categorical_features = [1,	4,	5,	6,	7,	8,	9,	10,	11,	12,	13,	14,	19,	20,	21,	22,	23,	25,	26,	27,	28,	29,	30,	31,	33,	37,	38,	39,	40,	51,	53,	55,	57,	60,	61,	62,	72,	73]
numeric_features = [0,	2,	3,	15,	16,	17,	18,	24,	32,	34,	35,	36,	41,	42,	43,	44,	45,	46,	47,	48,	49,	50,	52,	54,	56,	58,	59,	63,	64,	65,	66,	67,	68,	69,	70,	71]
mapping = {}

def binarize(file_list, new = True, sales = True, smart = False):
    y = []
    new_data = []    
    for row in file_list:
        new_row = []
    
        if sales:
            y.append(log(float(row[-1])))
            end = -1
        else:
            end = None
            
        for j, x in enumerate(row[1:end]):
            feature = (j, x)
            if new:
                if feature not in mapping:
                    mapping[feature] = len(mapping) # insert a new feature into the index
                new_row.append(mapping[feature])
            else:
                if feature in mapping:
                    new_row.append(mapping[feature])
        new_data.append(new_row)

    bindata = np.zeros((len(new_data),len(mapping)))
    for i, row in enumerate(new_data):
        for x in row:
            bindata[i][x] = 1

    return bindata, y




# BINARIZE DATA
bindata_train, train_y = binarize(train, new = True)
bindata_dev, dev_y = binarize(dev, new = False)
bindata_test, test_y = binarize(test, new = False, sales = False)


        
# CALL LINEAR REGRESSION, FIT MODEL, PREDICT MODEL
lm = linear_model.LinearRegression()
fit_mod = lm.fit(bindata_train, train_y)
pred_y = fit_mod.predict(bindata_dev)
print('Mean Squared Error for part 4 is:', sqrt(mean_squared_error(dev_y, pred_y)))
        
        




Mean Squared Error for part 4 is: 0.15630755195257634


In [5]:

# PART 2 QUESTION 5 TOP 10 AND BOTTOM 10 CURRENTLY NOT PULLING CORRECTLY, LIKELY BECAUSE OF POOR MODEL FIT

top_bott = []
for i, w in enumerate(fit_mod.coef_):
    top_bott.append((i,w))

top_bott_sorted = sorted(top_bott, key = lambda x: x[1], reverse = True)

    
    
for i in range(10):
    # POSITIVE
    for key, vals in mapping.items():
        if vals == top_bott_sorted[i][0]:
            feat_val = key
    print('\nPositive Index:', top_bott_sorted[i][0], 'Positive Weight:', top_bott_sorted[i][1], 'Positive Feature:', feat_val)
    
    neg_i = -i -1
    # NEGATIVE
    for key, vals in mapping.items():
        if vals == top_bott_sorted[neg_i][0]:
            feat_val = key
    print('Neg Index:', top_bott_sorted[neg_i][0], 'Neg Weight:', top_bott_sorted[neg_i][1], 'Neg Feature:', feat_val)


Positive Index: 45 Positive Weight: 0.1502200765778792 Positive Feature: (45, '1710')
Neg Index: 551 Neg Weight: -0.195700683334053 Neg Feature: (1, 'C (all)')

Positive Index: 299 Positive Weight: 0.1447142007193873 Positive Feature: (16, '9')
Neg Index: 35 Neg Weight: -0.18695065472452088 Neg Feature: (35, '0')

Positive Index: 8 Positive Weight: 0.12834144476475076 Positive Feature: (8, 'AllPub')
Neg Index: 0 Neg Weight: -0.16406225528777732 Neg Feature: (0, '60')

Positive Index: 311 Positive Weight: 0.12749464883943112 Positive Feature: (48, '3')
Neg Index: 37 Neg Weight: -0.14604398612052277 Neg Feature: (37, '856')

Positive Index: 168 Positive Weight: 0.12378709421335773 Positive Feature: (16, '8')
Neg Index: 1803 Neg Weight: -0.12858627347290982 Neg Feature: (45, '968')

Positive Index: 813 Positive Weight: 0.1205229242608985 Positive Feature: (11, 'StoneBr')
Neg Index: 3469 Neg Weight: -0.12769029207771548 Neg Feature: (67, '236')

Positive Index: 10 Positive Weight: 0.11893

In [6]:
# PART 2 QUESTION 6, BIAS DIMENSION
print('My fit intercept is:', fit_mod.intercept_)

My fit intercept is: 11.735119279699791


In [7]:
from math import exp
pred_test = fit_mod.predict(bindata_test)

exp_test = exp(1) ** pred_test
genIds = np.arange(1461,1461+len(exp_test))

df = pd.DataFrame({'Id': genIds, 'SalePrice': exp_test}).set_index('Id')

df.to_csv('predict_test.csv')

# 3: Smarter Binarization

In [8]:
#train_copy = [s.strip().split(",") for s in open('house-prices-advanced-regression-techniques\my_train_copy.csv').readlines()]
dev_copy = [s.strip().split(",") for s in open('house-prices-advanced-regression-techniques\my_dev_Copy.csv').readlines()]
dev_copy = dev_copy[1:]

train_copy = [s.strip().split(",") for s in open('house-prices-advanced-regression-techniques\my_train_Copy.csv').readlines()]
train_copy = train_copy[1:]

test_smart = [s.strip().split(",") for s in open(r'house-prices-advanced-regression-techniques\test_smart.csv').readlines()]
test_smart = test_smart[1:]

mapping = {}
def binarize_smart(file_list, new = True, sales = True):
    y = []
    new_data = []    
    for row in file_list:
        new_row = []
        row_cat = [row[i+1] for i in categorical_features] # row[1,2,4,7]
        if sales:
            y.append(log(float(row[-1])))
            end = -1
        else:
            end = None
            
        for j, x in enumerate(row_cat):
            feature = (j, x)
            if new:
                if feature not in mapping:
                    mapping[feature] = len(mapping) # insert a new feature into the index
                new_row.append(mapping[feature])
            else:
                if feature in mapping:
                    new_row.append(mapping[feature])
        new_data.append(new_row)

    bindata = np.zeros((len(new_data),len(mapping)))
    for i, row in enumerate(new_data):
        for x in row:
            bindata[i][x] = 1
            
            
    # CREATE NUMERIC ARRAY
    numeric_bindata_smart_train = np.zeros((len(bindata),len(numeric_features)))
    for i, row in enumerate(file_list):
        numer_row = [row[i+1] for i in numeric_features]
        for k, x in enumerate(numer_row):
            numeric_bindata_smart_train[i][k] = x 
    
    # CONCAT THE CATEGORICAL AND NUMERIC ARRAYS
    full_bindata_smart_train = np.zeros((len(new_data),len(mapping) + len(numeric_features)))

    full_bindata_smart_train = np.concatenate((numeric_bindata_smart_train,bindata), axis = 1)
            
    return full_bindata_smart_train, y

# CREATE CATEGORICAL ARRAY
bindata_smart_train, smart_train_y = binarize_smart(train_copy, new = True, sales = True)
bindata_smart_dev, smart_dev_y = binarize_smart(dev_copy, new = False, sales = True)
bindata_smart_test = binarize_smart(test_smart, new = False, sales = False)


        


lm2 = linear_model.LinearRegression()
fit_mod2 = lm.fit(bindata_smart_train, smart_train_y)
pred_y2 = fit_mod.predict(bindata_smart_dev)
print('Mean Squared Error for section 3 is:', sqrt(mean_squared_error(smart_dev_y, pred_y2)))

# PART 3 QUESTION 6, BIAS DIMENSION
print('My fit intercept is:', fit_mod2.intercept_)

Mean Squared Error for section 3 is: 0.12424584328249617
My fit intercept is: 10.65856288261491


In [9]:

# PART 3 QUESTION 5 TOP 10 AND BOTTOM 10 CURRENTLY NOT PULLING CORRECTLY, LIKELY BECAUSE OF POOR MODEL FIT

top_bott = []
for i, w in enumerate(fit_mod2.coef_):
    top_bott.append((i,w))

top_bott_sorted = sorted(top_bott, key = lambda x: x[1], reverse = True)

    
    
for i in range(10):
    # POSITIVE
    for key, vals in mapping.items():
        if vals == top_bott_sorted[i][0]:
            feat_val = key
    print('\nPositive Index:', top_bott_sorted[i][0], 'Positive Weight:', top_bott_sorted[i][1], 'Positive Feature:', feat_val)
    
    neg_i = -i -1
    # NEGATIVE
    for key, vals in mapping.items():
        if vals == top_bott_sorted[neg_i][0]:
            feat_val = key
    print('Neg Index:', top_bott_sorted[neg_i][0], 'Neg Weight:', top_bott_sorted[neg_i][1], 'Neg Feature:', feat_val)


Positive Index: 241 Positive Weight: 0.6118120187965786 Positive Feature: (14, 'CBlock')
Neg Index: 275 Neg Weight: -2.338997683125946 Neg Feature: (14, 'CBlock')

Positive Index: 215 Positive Weight: 0.46863921346666954 Positive Feature: (13, 'Tar&Grv')
Neg Index: 257 Neg Weight: -0.7289277992951565 Neg Feature: (13, 'Tar&Grv')

Positive Index: 260 Positive Weight: 0.40100490430077973 Positive Feature: (13, 'Tar&Grv')
Neg Index: 271 Neg Weight: -0.3939452474385067 Neg Feature: (13, 'Tar&Grv')

Positive Index: 272 Positive Weight: 0.35431883553260773 Positive Feature: (13, 'Tar&Grv')
Neg Index: 168 Neg Weight: -0.32776894758913616 Neg Feature: (20, 'Fa')

Positive Index: 228 Positive Weight: 0.32509770428142837 Positive Feature: (36, 'Oth')
Neg Index: 263 Neg Weight: -0.2259928954964688 Neg Feature: (36, 'Oth')

Positive Index: 274 Positive Weight: 0.30501096416332896 Positive Feature: (36, 'Oth')
Neg Index: 240 Neg Weight: -0.22251002280815738 Neg Feature: (25, 'Floor')

Positive Ind

# Part 4: Expermentation


In [10]:
# RIDGE REGRESSION NAIVE
from sklearn.linear_model import Ridge

for i in np.arange(4,6,0.1):
    clf = Ridge(alpha=i)
    fit_ridge_naive = clf.fit(bindata_train, train_y)

    #fit_mod = lm.fit
    pred_y = fit_ridge_naive.predict(bindata_dev)
    print(f'Mean Squared Error for ridge alpha {i} (naive) is:', round(sqrt(mean_squared_error(dev_y, pred_y)),4))
        

Mean Squared Error for ridge alpha 4.0 (naive) is: 0.1391
Mean Squared Error for ridge alpha 4.1 (naive) is: 0.1391
Mean Squared Error for ridge alpha 4.199999999999999 (naive) is: 0.1391
Mean Squared Error for ridge alpha 4.299999999999999 (naive) is: 0.1391
Mean Squared Error for ridge alpha 4.399999999999999 (naive) is: 0.1391
Mean Squared Error for ridge alpha 4.499999999999998 (naive) is: 0.1391
Mean Squared Error for ridge alpha 4.599999999999998 (naive) is: 0.1391
Mean Squared Error for ridge alpha 4.6999999999999975 (naive) is: 0.1391
Mean Squared Error for ridge alpha 4.799999999999997 (naive) is: 0.139
Mean Squared Error for ridge alpha 4.899999999999997 (naive) is: 0.139
Mean Squared Error for ridge alpha 4.9999999999999964 (naive) is: 0.139
Mean Squared Error for ridge alpha 5.099999999999996 (naive) is: 0.139
Mean Squared Error for ridge alpha 5.199999999999996 (naive) is: 0.139
Mean Squared Error for ridge alpha 5.299999999999995 (naive) is: 0.139
Mean Squared Error for r

In [11]:
# RIDGE SMART
for i in np.arange(1,6,0.5):
    clf = Ridge(alpha=i)
    fit_ridge_smart = clf.fit(bindata_smart_train, smart_train_y)

    #fit_mod = lm.fit
    pred_y_ridge = fit_ridge_smart.predict(bindata_smart_dev)
    print(f'Mean Squared Error for ridge alpha {i} (naive) is:', round( sqrt(mean_squared_error(smart_dev_y, pred_y_ridge)),4))
        


Mean Squared Error for ridge alpha 1.0 (naive) is: 0.1275
Mean Squared Error for ridge alpha 1.5 (naive) is: 0.1279
Mean Squared Error for ridge alpha 2.0 (naive) is: 0.1281
Mean Squared Error for ridge alpha 2.5 (naive) is: 0.1282
Mean Squared Error for ridge alpha 3.0 (naive) is: 0.1282
Mean Squared Error for ridge alpha 3.5 (naive) is: 0.1282
Mean Squared Error for ridge alpha 4.0 (naive) is: 0.1282
Mean Squared Error for ridge alpha 4.5 (naive) is: 0.1282
Mean Squared Error for ridge alpha 5.0 (naive) is: 0.1282
Mean Squared Error for ridge alpha 5.5 (naive) is: 0.1281


In [12]:
# Try anything else Part 4 question 5

from sklearn.linear_model import Lasso

for i in np.arange(1,6,0.5):
    lm_lasso = linear_model.LinearRegression()
    clf = Lasso(alpha=i)
    fit_lasso_naive = clf.fit(bindata_train, train_y)

    #fit_mod = lm.fit
    pred_y = fit_lasso_naive.predict(bindata_dev)
    print(f'Mean Squared Error for lasso alpha {i} (naive) is:', round(sqrt(mean_squared_error(dev_y, pred_y)),4))
    
    
for i in np.arange(1,6,0.5):
    lm_lasso = linear_model.LinearRegression()
    clf = Lasso(alpha=i)
    fit_lasso_naive = clf.fit(bindata_smart_train, smart_train_y)

    #fit_mod = lm.fit
    pred_y_ridge = fit_lasso_naive.predict(bindata_smart_dev)
    print(f'Mean Squared Error for lasso alpha {i} (smart) is:', round( sqrt(mean_squared_error(smart_dev_y, pred_y_ridge)),4))
        

Mean Squared Error for lasso alpha 1.0 (naive) is: 0.365
Mean Squared Error for lasso alpha 1.5 (naive) is: 0.365
Mean Squared Error for lasso alpha 2.0 (naive) is: 0.365
Mean Squared Error for lasso alpha 2.5 (naive) is: 0.365
Mean Squared Error for lasso alpha 3.0 (naive) is: 0.365
Mean Squared Error for lasso alpha 3.5 (naive) is: 0.365
Mean Squared Error for lasso alpha 4.0 (naive) is: 0.365
Mean Squared Error for lasso alpha 4.5 (naive) is: 0.365
Mean Squared Error for lasso alpha 5.0 (naive) is: 0.365
Mean Squared Error for lasso alpha 5.5 (naive) is: 0.365
Mean Squared Error for lasso alpha 1.0 (smart) is: 0.1593
Mean Squared Error for lasso alpha 1.5 (smart) is: 0.1634
Mean Squared Error for lasso alpha 2.0 (smart) is: 0.1671
Mean Squared Error for lasso alpha 2.5 (smart) is: 0.1712
Mean Squared Error for lasso alpha 3.0 (smart) is: 0.1767
Mean Squared Error for lasso alpha 3.5 (smart) is: 0.1797
Mean Squared Error for lasso alpha 4.0 (smart) is: 0.1805
Mean Squared Error for l

In [13]:
from math import exp
pred_test = fit_mod2.predict(bindata_smart_test[0])

exp_test = exp(1) ** pred_test
genIds = np.arange(1461,1461+len(exp_test))

df = pd.DataFrame({'Id': genIds, 'SalePrice': exp_test}).set_index('Id')

df.to_csv('predict_test.csv')

array([[   20.,    80., 11622., ...,     0.,     0.,     0.],
       [   20.,    81., 14267., ...,     0.,     0.,     0.],
       [   60.,    74., 13830., ...,     0.,     0.,     0.],
       ...,
       [   20.,   160., 20000., ...,     0.,     0.,     0.],
       [   85.,    62., 10441., ...,     0.,     0.,     0.],
       [   60.,    74.,  9627., ...,     0.,     0.,     0.]])